In [1]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'jinaai/jina-reranker-v2-base-multilingual',
    torch_dtype="auto",
    trust_remote_code=True,
)

model.to('cuda') # or 'cpu' if no GPU is available
model.eval()


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768)
      (token_type_embeddings): Embedding(1, 768)
    )
    (emb_drop): Dropout(p=0.1, inplace=False)
    (emb_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): XLMRobertaEncoder(
      (layers): ModuleList(
        (0-11): 12 x Block(
          (mixer): MHA(
            (Wqkv): LinearResidual(in_features=768, out_features=2304, bias=True)
            (inner_attn): FlashSelfAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (inner_cross_attn): FlashCrossAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (dropout1): Dropout(p=0.1, inplace=False)
          (drop_path1): StochasticDepth(p=0

In [7]:
# Example query and documents
query = "I- Ubiquité et diversité des micro organismes"
documents = [
    "lors d’un effort physique, les rythmes respiratoire et cardiaque augmentent pour apporter plus de sang riche en dioxygène au niveau des muscles sollicités",
    "Relier la présence de micro-organismes dans le tube digestif à certaines caractéristiques de la digestion.",
    "Ce thème se prête aux applications biotechnologiques, lorsque l’élève réalise des cultures de cellules ou étudie des protocoles d’obtention d’organismes génétiquement modifiés: OGM",
    "Mettre en relation les mouvements des plaques de lithosphère sur l’asthénosphère, également solide mais moins rigide avec séismes et éruptions volcaniques.",
    "Associer faille, séisme et mouvements de blocs rocheux et expliquer qu’ils témoignent de l’accumulation de tensions liées au mouvement des plaques lithosphériques.",
    "Associer le volcanisme, essentiellement explosif, aux zones de convergence lithosphérique (fosses océaniques) et le volcanisme, essentiellement effusif, aux zones de divergence (dorsales océaniques).",
    "Relier la tectonique des plaques à la dissipation de l’énergie thermique d’origine interne.",
    "Distinguer ce qui relève d’un phénomène météorologique et ce qui relève d’un phénomène climatique.",
    "Expliquer à l’échelle globale que les mouvements des masses d’air et des masses d’eau à l’origine des phénomènes météorologiques, et les grandes zones climatiques, sont en relation avec l’inégale distribution du rayonnement solaire à la surface de la planète.",
    "Identifier le couplage entre les mouvements des masses d’air (vents) et des masses d’eau (courants océaniques) et ses effets sur les climats.",
    "Repérer au moins un changement climatique passé (temps géologique) et ses origines possibles.",
    "Expliquer le réchauffement climatique actuel (influence des activités humaines sur le climat) et en envisager les effets à long terme.",
    "Mettre en relation un phénomène naturel (aléa) avec les enjeux présents sur une zone géographique déterminée, leur vulnérabilité et ainsi identifier et caractériser un risque.",
    "Identifier des mesures de prévention, de protection, d’adaptation ou d’atténuation en relation avec un risque.",
    "Expliquer ces mesures et argumenter des choix de comportements individuel et collectif responsables en matière de risque naturel.",
    "Expliquer ce que la Terre a de spécifique et ce qu’elle partage avec différents objets du système solaire.",
    "Expliquer le rôle majeur du Soleil sur certaines des caractéristiques des planètes telluriques et gazeuses.",
    "Articuler la notion d’ères géologiques avec différents évènements géologiques et biologiques survenus sur Terre.",
    "Caractériser quelques grands enjeux (aux niveaux régional et mondial) de l’exploitation de ressources naturelles renouvelables et non renouvelables en lien avec les besoins en nourriture et les activités humaines.",
    "Relier la formation de ressources naturelles et différentes manifestations de l’activité du globe.",
    "Relier l’exploitation des ressources naturelles (gisement- gestion-renouvellement ou pas) et ses impacts à différentes échelles.",
    "Relier la vitesse de la production de biomasse et/ou de la formation des gisements à leur exploitation raisonnée.",
    "Expliquer les conflits d’usage ou d’exploitation pour quelques exemples de ressources naturelles.",
    "Identifier et caractériser des modifications, au cours du temps, de l’organisation et du fonctionnement de quelques écosystèmes en lien avec certaines actions humaines.",
    "Mettre en relation certaines activités humaines avec la biodiversité des écosystèmes et leurs dynamiques.",
    "Évaluer quelques effets des activités humaines en termes de bénéfices-risques pour les écosystèmes et pour les êtres humains.",
    "Relier le fonctionnement des écosystèmes au cours du temps à des mesures de d’atténuation, de prévention ou de réhabilitation.",
    "Expliquer ces mesures et argumenter des choix de comportements individuel et collectif responsables en matière de protection environnementale.",
    "Expliquer la transformation des aliments en nutriments lors de la digestion, sous l’action d’enzymes et le passage des nutriments vers le milieu intérieur.",
    "Relier des systèmes digestifs à des régimes alimentaires (phytophages ; zoophages).",
    "Expliquer que les cellules animales utilisent de la matière organique et de la matière minérale pour produire leur propre matière organique.",
    "Relier le passage du dioxygène des milieux de vie au niveau des appareils respiratoires aux caractéristiques des surfaces d’échanges.",
    "Relier les systèmes de transport (appareil circulatoire endigué ou non ; milieu intérieur) aux lieux d’utilisation et de stockage des nutriments (besoins des cellules ; tissus de stockage).",
    "Relier les systèmes de transport et l’élimination des déchets produits au cours du fonctionnement cellulaire.",
    "Relier la présence de micro-organismes dans le tube digestif à certaines caractéristiques de la digestion.",
    "Expliquer l’approvisionnement des cellules chlorophylliennes en eau, en sels minéraux et en dioxyde de carbone, pour satisfaire ses besoins nutritifs, en reliant les lieux de prélèvement et les systèmes de transport dans le végétal (circulation de la sève brute dans des vaisseaux conducteurs).",
    "Relier la production de matière organique au niveau des cellules chlorophylliennes des feuilles à l’utilisation de lumière et de matière minérale (photosynthèse) et les lieux d’utilisation et de stockage (circulation de la sève élaborée dans des vaisseaux conducteurs).",
    "Relier l’énergie nécessaire au fonctionnement des cellules animale et végétale à l’utilisation de dioxygène et de glucose.",
    "Expliquer que la nutrition minérale implique la symbiose avec des micro-organismes du sol.",
    "Identifier des caractères propres à une espèce et distinguer un caractère des formes variables qu’il peut prendre chez les individus d’une même espèce : génotype et phénotype ; influence de l’environnement sur le phénotype.",
    "Expliquer que toutes les cellules d’un individu (à l’exception des gamètes) possèdent le même nombre de chromosomes par noyau à l’issue de la mitose.",
    "Relier l’ADN des chromosomes au support de l’information génétique.",
    "Relier l’apparition de nouveaux allèles à l’existence de mutations.",
    "Expliquer la diversité et l’héritabilité de caractères par le brassage de l’information génétique associé à la méiose et à la fécondation.",
    "Repérer et relier la biodiversité aux différentes échelles du vivant (écosystème, espèces et allèles).",
    "Mettre en relation les modifications de la biodiversité au cours des temps géologiques avec des faits montrant l’évolution des groupes d’êtres vivants (apparition, disparition, diversification et raréfaction).",
    "Exploiter les traces fossiles permettant d’identifier les premiers organismes sur Terre.",
    "Expliquer l’évolution des espèces par des processus de sélection naturelle en mettant en relation les caractéristiques phénotypiques d’organismes issus du hasard de la reproduction avec des conditions qui les rendent plus aptes à se reproduire.",
    "Argumenter le degré de parenté entre des organismes actuels et/ou fossiles à partir des caractères partagés.",
    "Argumenter l’histoire évolutive de grands groupes d’êtres vivants, dont Homo sapiens, en exploitant différents faits (caractères des espèces actuelles et fossiles ; liens de parenté entre espèces actuelles et/ou fossiles ; données sur les paléo milieux de vie).",
    "Relier certaines modalités de la reproduction sexuée (oviparité/viviparité ; fécondation externe/interne ; reproduction des plantes à fleurs) aux pressions exercées par les milieux.",
    "Expliquer la stabilité et la diversité des phénotypes des individus d’une population par les mécanismes de la reproduction sexuée (production de gamètes apportant la moitié du patrimoine génétique de l’espèce et fécondation).",
    "Identifier des modes de reproduction asexuée.",
    "Relier la reproduction asexuée à une stabilité des phénotypes entre générations.",
    "Relier les modes de reproduction (sexuée et asexuée), les conditions du milieu (rareté ou abondance des ressources alimentaires, des prédateurs, des conditions physicochimiques, etc.) à la dynamique des populations."
]
# construct sentence pairs
sentence_pairs = [[query, doc] for doc in documents]

scores = model.compute_score(sentence_pairs, max_length=1024)


sorted_scores = sorted(zip(scores, documents), reverse=True)

for score, doc in sorted_scores:
    print(doc)
    print(score)
    print("============")


Distinguer ce qui relève d’un phénomène météorologique et ce qui relève d’un phénomène climatique.
0.28378137946128845
Repérer au moins un changement climatique passé (temps géologique) et ses origines possibles.
0.188323974609375
Relier le fonctionnement des écosystèmes au cours du temps à des mesures de d’atténuation, de prévention ou de réhabilitation.
0.13206614553928375
Identifier des mesures de prévention, de protection, d’adaptation ou d’atténuation en relation avec un risque.
0.13028541207313538
Expliquer ces mesures et argumenter des choix de comportements individuel et collectif responsables en matière de protection environnementale.
0.13028541207313538
Relier l’énergie nécessaire au fonctionnement des cellules animale et végétale à l’utilisation de dioxygène et de glucose.
0.1233656257390976
Identifier le couplage entre les mouvements des masses d’air (vents) et des masses d’eau (courants océaniques) et ses effets sur les climats.
0.1225232183933258
Relier la formation de re

## Test SIReranker

In [1]:
import sys
sys.path.insert(0, '../app')
sys.path.insert(0, '../')
from app.processing.text.SIReranker import SIReranker


reranker = SIReranker()

/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:182: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):


In [2]:

import sys
import os
sys.path.insert(0, '../')
sys.path.insert(0, '../app')


from pydantic import BaseModel
import weaviate
from app.search.search_chunks import search_all_chunks

query = "dinosaures"
client = weaviate.connect_to_local()
chunks = search_all_chunks(client=client, query=query).chunks
client.close()

reranker.sort_docment_chunks(document_chunks=chunks, query=query)


ALL PROPS ['uuid', 'document_id', 'public_path', 'original_path', 's3_object_name', 'user_approved', 'user_disapproved', 'media_name', 'document', 'text', 'title', 'media_type', 'meta_start', 'meta_end', 'meta_s3_object_name', 'meta_page_number', 'meta_bbox', 'meta_s3_object_name', 'meta_page_number', 'meta_bbox', 'meta_question', 'meta_answer', 'meta_url', 'meta_description', 'meta_title', 'meta_type', 'meta_url']
QUERY dinosaures


[ChunkWithScore(uuid='db86e058-bfbc-41d9-8b0f-8dfb0e77a8b0', document=Document(uuid='32782090-acb0-44b6-9604-b6239a561aa2', document_id='70cc4c0c-7d44-4ac7-a241-a0fb8820e380', public_path='', original_path='/home/erwan/Desktop/clients/ScienceInfuse/server/ftp-data/2015 - Dinosaures.pdf', s3_object_name='pdf/70cc4c0c-7d44-4ac7-a241-a0fb8820e380.pdf', user_approved=None, user_disapproved=None, media_name='2015 - Dinosaures'), text='Le terme dinosaure est forgé à partir de deux mots du grec ancien : δεινός (deinόs, terrible) et σαῦρος (saûros, lézard). Ce nom est toutefois quelque peu trompeur car les dinosaures ne sont pas des lézards ; ils représentent un clade distinct de reptiles apparus au cours de la période Trias il y a 230 millions d’années. Rappelons qu’un clade est un groupe d’organismes vivants ou ayant vécu comprenant un organisme particulier et la totalité de ses descendants.', title='II.2 Un mot sur les échelles de temps géologiques>II.3 Généralités sur les dinosaures', medi